In [1]:
import SimpleITK as sitk
import numpy as np
import csv
import scipy
from glob import glob
import pandas as pd
from scipy import ndimage
from tqdm import tqdm 
import pandas as pd
import skimage, os
from skimage.morphology import ball, disk, dilation, binary_erosion, remove_small_objects, erosion, closing, reconstruction, binary_closing
from skimage.measure import label,regionprops, perimeter
from skimage.morphology import binary_dilation, binary_opening
from skimage.filters import roberts, sobel
from skimage import measure, feature
from skimage.segmentation import clear_border
from skimage import data
from sklearn.preprocessing import MinMaxScaler
from scipy import ndimage
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import dicom
import scipy.misc
import numpy as np
from skimage import measure, morphology, segmentation
import matplotlib.pyplot as plt
from joblib import Parallel, delayed
from numba import autojit
from PIL import Image
import cv2
import os

from utils_3d import *
from paths import *

%matplotlib inline

Using TensorFlow backend.


In [2]:
#min_max_scaler.fit_transform(X_train)

In [3]:
# In[7]:

def load_train():
    data_path = src
    folders = [x for x in os.listdir(data_path) if 'subset' in x]
    os.chdir(data_path)
    patients = []
    
    for i in folders:
        os.chdir(data_path + i)
        #print('Changing folder to: {}'.format(data_path + i))
        patient_ids = [x for x in os.listdir(data_path + i) if '.mhd' in x]
        for id in patient_ids:
            j = '{}/{}'.format(i, id)
            patients.append(j)
    return patients

def get_filename(file_list, case):
    for f in file_list:
        if case in f:
            return(f)
                
def get_pixels_hu(image):
    image = image.astype(np.int16)
    image[image <= -1200] = -1200
    image[image >= 600] = 600
    min_max_scaler = MinMaxScaler()
    min_max_scaler = MinMaxScaler(feature_range=(0,255))
    for i in range(image.shape[0]):
        image[i] = min_max_scaler.fit_transform(image[i])    
    return np.array(image, dtype=np.int16)

def seq(start, stop, step=1):
    n = int(round((stop - start)/float(step)))
    if n > 1:
        return([start + step*i for i in range(n+1)])
    else:
        return([])
        
def load_itk(filename):
    itkimage = sitk.ReadImage(filename)
    numpyImage = sitk.GetArrayFromImage(itkimage)
    numpyOrigin = np.array(list(reversed(itkimage.GetOrigin())))
    numpySpacing = np.array(list(reversed(itkimage.GetSpacing())))
    return numpyImage, numpyOrigin, numpySpacing

def world_2_voxel(world_coordinates, origin, spacing):
    stretched_voxel_coordinates = np.absolute(world_coordinates - origin)
    voxel_coordinates = stretched_voxel_coordinates / spacing
    return voxel_coordinates

def voxel_2_world(voxel_coordinates, origin, spacing):
    stretched_voxel_coordinates = voxel_coordinates * spacing
    world_coordinates = stretched_voxel_coordinates + origin
    return world_coordinates


# In[9]:

def generate_markers(image):
    #Creation of the internal Marker
    marker_internal = image < -400
    marker_internal = segmentation.clear_border(marker_internal)
    marker_internal_labels = measure.label(marker_internal)
    areas = [r.area for r in measure.regionprops(marker_internal_labels)]
    areas.sort()
    if len(areas) > 2:
        for region in measure.regionprops(marker_internal_labels):
            if region.area < areas[-2]:
                for coordinates in region.coords:                
                       marker_internal_labels[coordinates[0], coordinates[1]] = 0
    marker_internal = marker_internal_labels > 0
    #Creation of the external Marker
    external_a = ndimage.binary_dilation(marker_internal, iterations=10)
    external_b = ndimage.binary_dilation(marker_internal, iterations=55)
    marker_external = external_b ^ external_a
    #Creation of the Watershed Marker matrix
    marker_watershed = np.zeros(image.shape, dtype=np.int)
    marker_watershed += marker_internal * 255
    marker_watershed += marker_external * 128
    return marker_internal, marker_external, marker_watershed


def get_segmented_lungs(image):
    #Creation of the markers as shown above:
    marker_internal, marker_external, marker_watershed = generate_markers(image)
    
    #Creation of the Sobel-Gradient
    sobel_filtered_dx = ndimage.sobel(image, 1)
    sobel_filtered_dy = ndimage.sobel(image, 0)
    sobel_gradient = np.hypot(sobel_filtered_dx, sobel_filtered_dy)
    sobel_gradient *= 255.0 / np.max(sobel_gradient)
    
    #Watershed algorithm
    watershed = morphology.watershed(sobel_gradient, marker_watershed)
    
    #Reducing the image created by the Watershed algorithm to its outline
    outline = ndimage.morphological_gradient(watershed, size=(3,3))
    outline = outline.astype(bool)
    
    #Performing Black-Tophat Morphology for reinclusion
    #Creation of the disk-kernel and increasing its size a bit
    blackhat_struct = [[0, 0, 1, 1, 1, 0, 0],
                       [0, 1, 1, 1, 1, 1, 0],
                       [1, 1, 1, 1, 1, 1, 1],
                       [1, 1, 1, 1, 1, 1, 1],
                       [1, 1, 1, 1, 1, 1, 1],
                       [0, 1, 1, 1, 1, 1, 0],
                       [0, 0, 1, 1, 1, 0, 0]]
    #blackhat_struct = ndimage.iterate_structure(blackhat_struct, 8)
    blackhat_struct = ndimage.iterate_structure(blackhat_struct, 14) # <- retains more of the area, 12 works well. Changed to 14, 12 still excluded some parts.
    #Perform the Black-Hat
    outline += ndimage.black_tophat(outline, structure=blackhat_struct)
    
    #Use the internal marker and the Outline that was just created to generate the lungfilter
    lungfilter = np.bitwise_or(marker_internal, outline)
    #Close holes in the lungfilter
    #fill_holes is not used here, since in some slices the heart would be reincluded by accident
    lungfilter = ndimage.morphology.binary_closing(lungfilter, structure=np.ones((5,5)), iterations=3)
    
    #Apply the lungfilter (note the filtered areas being assigned threshold_min HU)
    segmented = np.where(lungfilter == 1, image, threshold_min*np.ones(image.shape))
    
    #return segmented, lungfilter, outline, watershed, sobel_gradient, marker_internal, marker_external, marker_watershed
    return segmented


def segment_lung_from_ct_scan(ct_scan):
    return np.asarray([get_segmented_lungs(slice) for slice in ct_scan])


def get_nodule_slices(lung_mask, nodule_mask, lung_raw):
    indexes = []
    for img_slice in range(nodule_mask.shape[0]):
        if np.max(nodule_mask[img_slice]) == 1.0:
            indexes.append(img_slice)

    #print('Nodule_present on slices: {}'.format(indexes))
    lung_mask_pres = lung_mask[indexes, :, :]
    nod_mask_pres = nodule_mask[indexes, :, :]
    lung_raw_pres = lung_raw[indexes, :, :]
    return lung_mask_pres, nod_mask_pres, lung_raw_pres

def reshape_3d(image_3d):
    reshaped_img = image_3d.reshape([image_3d.shape[0], 1, height_mask, width_mask])
    #print('Reshaped image shape:', reshaped_img.shape)
    return reshaped_img

In [4]:
def create_masks_for_patient_watershed(img_file, save = True):    
    def draw_nodule_mask(node_idx, cur_row, lung_img):
        #print('Working on node: {}, row: {}'.format(node_idx, cur_row), '\n')
        coord_x = cur_row["coordX"]
        coord_y = cur_row["coordY"]
        coord_z = cur_row["coordZ"]
        diam = cur_row["diameter_mm"]
        radius = np.ceil(diam/2)

        world_center = np.array((coord_z,coord_y,coord_x))   # nodule center
        voxel_center = world_2_voxel(world_center, origin, new_spacing)
        zz = int(voxel_center[0])
        yy = int(voxel_center[1])
        xx = int(voxel_center[2])

        image_mask = np.zeros([64,64,64])
        lung_cube = np.zeros([64,64,64])
        
        if zz < 32 or 
        lung_cube[:,:,:] = lung_img[zz-32:zz+32,yy-32:yy+32,xx-32:xx+32]
        
        image_mask[zz-int(radius):zz+int(radius),yy-int(radius):yy+int(radius),xx-int(radius):xx+int(radius)] = int(1)

        return image_mask,lung_cube


    #print("Getting mask for image file {}".format(img_file))
    patient_id = img_file.split('/')[-1][:-4]
    mini_df = df_node[df_node["file"] == img_file]

    if mini_df.shape[0] > 0: # some files may not have a nodule--skipping those 

        img, origin, spacing = load_itk(src + img_file)
        height, width = img.shape[1], img.shape[2]

        #calculate resize factor
        RESIZE_SPACING = [1, 1, 1]
        resize_factor = spacing / RESIZE_SPACING
        new_real_shape = img.shape * resize_factor
        new_shape = np.round(new_real_shape)
        real_resize = new_shape / img.shape
        new_spacing = spacing / real_resize
        lung_img = scipy.ndimage.interpolation.zoom(img, real_resize)       
        lung_img = get_pixels_hu(lung_img)       
        
        lung_mask = lung_img.copy()

        for node_idx, cur_row in tqdm(mini_df.iterrows(),total=len(mini_df)): 
            nodule_mask,lung_cube = draw_nodule_mask(node_idx, cur_row,lung_img)
        

    
        np.save('{}/lung_mask/{}'.format(dst_nodules, patient_id), lung_cube)
        np.save('{}/nodule_mask/{}'.format(dst_nodules, patient_id), nodule_mask)


        return
    
    else: 
        #print('\n', 'No nodules found for patient: {}'.format(patient_id), '\n')
        return

In [5]:
annotations_path = annotations_train_path
src = src_train
dst_nodules = mask_val

In [6]:
patients = load_train()
df_node = pd.read_csv(annotations_path+"annotations.csv")
df_node["file"] = df_node["seriesuid"].map(lambda file_name: get_filename(patients, file_name))
df_node = df_node.dropna()

In [7]:
Parallel(n_jobs=-1)(delayed(create_masks_for_patient_watershed)(patient) for patient in sorted(patients[0:4]))

100%|██████████| 2/2 [00:00<00:00, 383.88it/s]


JoblibValueError: JoblibValueError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
/Users/mahui/anaconda/lib/python2.7/runpy.py in _run_module_as_main(mod_name='ipykernel_launcher', alter_argv=1)
    169     pkg_name = mod_name.rpartition('.')[0]
    170     main_globals = sys.modules["__main__"].__dict__
    171     if alter_argv:
    172         sys.argv[0] = fname
    173     return _run_code(code, main_globals, None,
--> 174                      "__main__", fname, loader, pkg_name)
        fname = '/Users/mahui/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py'
        loader = <pkgutil.ImpLoader instance>
        pkg_name = ''
    175 
    176 def run_module(mod_name, init_globals=None,
    177                run_name=None, alter_sys=False):
    178     """Execute a module's code without importing it

..........................................